In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from tqdm import tqdm_notebook as tqdm
import string

In [ ]:
df = pd.read_csv("results.csv", error_bad_lines=False, encoding='Latin-1', warn_bad_lines=False);
df.head()

In [ ]:
df = df.drop(columns=["synset_type", "synset_id", "image_id"])
df.head()

In [ ]:
urls = df["image_url"].values
del df
urls

In [ ]:
from PIL import Image
import requests
from io import BytesIO

response = requests.get(urls[-3])
img = Image.open(BytesIO(response.content))

In [ ]:
plt.imshow(np.asarray(img))

In [ ]:
img.save("images/test.jpg")

In [9]:
img.save("images/"+str(0)+".jpg")


In [10]:
urls.shape

(14265468,)

In [11]:
import ray
ray.init()

2020-12-22 14:25:58,475	INFO services.py:1090 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.141',
 'raylet_ip_address': '192.168.1.141',
 'redis_address': '192.168.1.141:6379',
 'object_store_address': '/tmp/ray/session_2020-12-22_14-25-57_968108_19845/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-12-22_14-25-57_968108_19845/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2020-12-22_14-25-57_968108_19845',
 'metrics_export_port': 64489,
 'node_id': 'b0ce6b33e97d02ade58ab6d9cb28a50fb8e705bd'}

In [12]:
clean_urls = set()

counter = 0
@ray.remote
def process_image(url):
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        name = url.translate(str.maketrans('', '', string.punctuation))
        img.save("images/"+name+".jpg")
        return
    except Exception as e:
        #print(e)
        return
    

In [ ]:
[process_image.remote(url) for url in tqdm(urls[:int(len(urls)/3)])]
    

In [13]:
[process_image.remote(url) for url in tqdm(urls[int(len(urls)/3):int(2*len(urls)/3)])]


<ipython-input-13-a78352943704>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  [process_image.remote(url) for url in tqdm(urls[int(len(urls)/3):int(2*len(urls)/3)])]


[ObjectRef(df5a1a828c9685d3ffffffff0100000001000000),
 ObjectRef(cb230a572350ff44ffffffff0100000001000000),
 ObjectRef(7bbd90284b71e599ffffffff0100000001000000),
 ObjectRef(bd37d2621480fc7dffffffff0100000001000000),
 ObjectRef(88866c7daffdd00effffffff0100000001000000),
 ObjectRef(d251967856448cebffffffff0100000001000000),
 ObjectRef(3bf0c856ace5a4d8ffffffff0100000001000000),
 ObjectRef(72e11b46e93d91e4ffffffff0100000001000000),
 ObjectRef(62223d85d5e7cd76ffffffff0100000001000000),
 ObjectRef(3106d80c4e3c2369ffffffff0100000001000000),
 ObjectRef(ae935fc0cb63c7d2ffffffff0100000001000000),
 ObjectRef(3db7cfefbaf00fd0ffffffff0100000001000000),
 ObjectRef(a628090ab940bf96ffffffff0100000001000000),
 ObjectRef(fafba2bafaed5dc3ffffffff0100000001000000),
 ObjectRef(b7603b6cc81d46c1ffffffff0100000001000000),
 ObjectRef(a49175458b016c38ffffffff0100000001000000),
 ObjectRef(84b65a94b9610224ffffffff0100000001000000),
 ObjectRef(87b4f726eda7082bffffffff0100000001000000),
 ObjectRef(be3cb809d884a562f

In [ ]:
[process_image.remote(url) for url in tqdm(urls[int(2*len(urls)/3):])]


<ipython-input-14-e0ef2e32dedd>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  [process_image.remote(url) for url in tqdm(urls[int(2*len(urls)/3):])]
